In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
from PIL import Image


In [28]:
def load_dataset(path):
    images = []
    labels = []
    class_names = os.listdir(path)
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(path, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = Image.open(image_path).convert('RGB')
            image = image.resize((32, 32))  # Resize the image to a fixed size
            image = np.array(image).flatten()  # Flatten the image
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

# Provide the path to your dataset
dataset_path = "H:/15 classes 6k Images"
images, labels = load_dataset(dataset_path)


In [29]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)


In [33]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for grid search
param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11, 13, 15]  # List of values to search
}

# Perform grid search
grid_search = GridSearchCV(knn_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_n_neighbors = grid_search.best_params_['knn__n_neighbors']
print("Best n_neighbors:", best_n_neighbors)

# Use the best hyperparameters to build and train the final KNN model
knn_model = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=best_n_neighbors))
])
knn_model.fit(X_train, y_train)

# Evaluate the accuracy of the final model
y_pred = knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best n_neighbors: 3
Accuracy: 0.5622222222222222


In [34]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((32, 32))  # Resize the image to a fixed size
    image = np.array(image).flatten()  # Flatten the image
    return image

# Provide the path to your test image
test_image_path = "H:\\15 classes 6k Images\\No rickshaws\\norickshaws1_output_0_883.png"
test_image = preprocess_image(test_image_path)

# Make a prediction on the test image
predicted_label = knn_model.predict([test_image])[0]

# Get the class label name corresponding to the predicted label
class_names = ['No left turn', 'No Overtaking', 'NO Parking', 'No rickshaws', 'No right turn', 'No trucks', 'No U turn', 'No use of horn', 'One way street', 'Parking', 'Pedestrian Crossing', 'School', 'Sharp bend to the right ahead', 'Side road on the left ahead', 'U-turn']
predicted_class_name = class_names[predicted_label]

print("Predicted class:", predicted_class_name)


Predicted class: One way street


In [35]:
# Debugging outputs
print("Test image path:", test_image_path)
print("Test image shape:", test_image.shape)
print("Flattened image shape:", test_image.flatten().shape)
print("Predicted label index:", predicted_label)


Test image path: H:\15 classes 6k Images\No rickshaws\norickshaws1_output_0_883.png
Test image shape: (3072,)
Flattened image shape: (3072,)
Predicted label index: 8
